# Figuring out the SDSS imaging pipeline 
to better understand observations and trying to be an astronomer...

In [1]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline 

from ChangTools.plotting import prettyplot
from ChangTools.plotting import prettycolors
from ChangTools.fitstables import mrdfits

In [2]:
from pydl.pydlutils.spheregroup import spherematch

/usr/local/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


## iSEDfit catalog

In [3]:
isedfit = mrdfits('../local/iGalPros/NSA_iSEDfit/nsa_v1_2_fsps_v2.4_miles_chab_charlot_sfhgrid01.fits.gz')

In [4]:
print sorted(isedfit.__dict__.keys())

['age', 'age_50', 'age_avg', 'age_err', 'av', 'av_50', 'av_avg', 'av_err', 'b100', 'b1000', 'b1000_50', 'b1000_avg', 'b1000_err', 'b100_50', 'b100_avg', 'b100_err', 'bestmaggies', 'bursttype', 'chi2', 'chunkindx', 'dec', 'delayed', 'dtburst', 'ewniiha', 'ewniiha_50', 'ewniiha_avg', 'ewniiha_err', 'ewoii', 'ewoii_50', 'ewoii_avg', 'ewoii_err', 'ewoiiihb', 'ewoiiihb_50', 'ewoiiihb_avg', 'ewoiiihb_err', 'fburst', 'isedfit_id', 'ivarmaggies', 'maggies', 'modelindx', 'mstar', 'mstar_50', 'mstar_avg', 'mstar_err', 'mu', 'mu_50', 'mu_avg', 'mu_err', 'nburst', 'nlyc', 'oiiihb', 'oiiihb_50', 'oiiihb_avg', 'oiiihb_err', 'ra', 'sfr', 'sfr100', 'sfr100_50', 'sfr100_avg', 'sfr100_err', 'sfr_50', 'sfr_avg', 'sfr_err', 'sfrage', 'sfrage_50', 'sfrage_avg', 'sfrage_err', 'tau', 'tau_50', 'tau_avg', 'tau_err', 'tburst', 'totalmass', 'totalmass_err', 'trunctau', 'z', 'zmetal', 'zmetal_50', 'zmetal_avg', 'zmetal_err']


## SDSS VAGC imaging catalog

In [5]:
sdss = mrdfits('../local/iGalPros/catalogs/object_sdss_imaging.fits')

In [6]:
print sorted(sdss.__dict__.keys())

['balkan_id', 'calib_status', 'calibobj_position', 'camcol', 'colc', 'dec', 'devflux', 'devflux_ivar', 'expflux', 'expflux_ivar', 'extinction', 'fiberflux', 'field', 'flags', 'flags2', 'fracpsf', 'id', 'ifield', 'modelflux', 'modelflux_ivar', 'nchild', 'nmgypercount', 'objc_colc', 'objc_flags', 'objc_flags2', 'objc_rowc', 'petroflux', 'petroflux_ivar', 'petror50', 'petror90', 'psfflux', 'psfflux_ivar', 'psp_skyflux', 'ra', 'rerun', 'resolve_status', 'run', 'score', 'sdss_imaging_tag', 'sdss_imaging_tag_primary', 'skyflux', 'vagc_select']


## NSA catalog

In [7]:
nsa = mrdfits('../local/iGalPros/catalogs/nsa_v0_1_2.fits')
print sorted(nsa.__dict__.keys())

['absmag', 'ahdew', 'ahdewerr', 'ahgew', 'ahgewerr', 'aid', 'amivar', 'asymmetry', 'av', 'b1000', 'b300', 'ba50', 'ba90', 'bastokes', 'camcol', 'clumpy', 'coeff', 'd4000', 'd4000err', 'dec', 'deccat', 'dflags', 'dversion', 'extinction', 'fa', 'faerr', 'fiberflux', 'fiberflux_ivar', 'fiberid', 'field', 'haew', 'haewerr', 'haflux', 'hafluxerr', 'havmeas', 'havmerr', 'hbew', 'hbewerr', 'hbflux', 'hbfluxerr', 'hbvmeas', 'hbvmerr', 'ialfalfa', 'iauname', 'ined', 'isdss', 'isixdf', 'itwodf', 'izcat', 'kcoeff', 'kcorrect', 'mag', 'mass', 'mets', 'mjd', 'mtol', 'n2ew', 'n2ewerr', 'n2flux', 'n2fluxerr', 'n2vmeas', 'n2vmerr', 'ne3ew', 'ne3ewerr', 'ne5ew', 'ne5ewerr', 'nmgy', 'nmgy_ivar', 'nprof', 'nsaid', 'o1ew', 'o1ewerr', 'o1flux', 'o1fluxerr', 'o1vmeas', 'o1vmerr', 'o2ew', 'o2ewerr', 'o2flux', 'o2fluxerr', 'o2vmeas', 'o2vmerr', 'o3ew', 'o3ewerr', 'o3flux', 'o3fluxerr', 'o3vmeas', 'o3vmerr', 'objno', 'ok', 'petroflux', 'petroflux_ivar', 'petroth50', 'petroth90', 'petrotheta', 'phi50', 'phi90',

## Question 1: 
How are the maggies from iSEDfit processed from VAGC? We care because iSEDfit maggies are what's used by iSEDfit to fit photometry. 

Here's how we're going to answer this question, 
- Randomly select iSEDfit galaxies
- Cross-match them in VAGc
- Put VAGC photometry into expected pipeline
- Compare output from pipeline to iSEDfit maggies

In [8]:
# Random iSEDfit galaxies
ised_ids = [53999, 18965, 81806]
ised_gals = [{} for i in range(len(ised_ids))]

for i, gal_dict in enumerate(ised_gals): 
    for key in isedfit.__dict__.keys(): 
        gal_dict[key] = getattr(isedfit, key)[ised_ids[i]]

In [9]:
# cross-match using spherematch onto SDSS VAGC galaxies
match = spherematch(sdss.ra, sdss.dec, # SDSS VAGC
                    np.array([gd['ra'] for gd in ised_gals]), np.array([gd['dec'] for gd in ised_gals]),  # random galaxies
                    0.000833333) # 3 arcsec 

# ... and check that its reasonable
for i_g in range(len(match[0])): 
    assert np.abs(sdss.ra[match[0][i_g]] - ised_gals[match[1][i_g]]['ra']) < 0.01
    assert np.abs(sdss.dec[match[0][i_g]] - ised_gals[match[1][i_g]]['dec']) < 0.01

In [22]:
print match[0], list(match[1]).index(0)

 [1903777  260452 1275710] 2


In [23]:
# store cross-matched SDSS VAGC imaging data onto ised dictionary
for i, gal_dict in enumerate(ised_gals): 
    sdss_dict = {} 
    for key in sdss.__dict__.keys(): 
        sdss_dict[key] = getattr(sdss, key)[match[0][list(match[1]).index(i)]]
    
    gal_dict['sdss_dict'] = sdss_dict

In [11]:
# cross-match using spherematch onto NSA catalog
match_nsa = spherematch(nsa.ra, nsa.dec, # NSA catalog
                        np.array([gd['ra'] for gd in ised_gals]), np.array([gd['dec'] for gd in ised_gals]),  # random galaxies
                        0.000833333) # 3 arcsec

In [26]:
# store cross-matched NSA data onto ised dictionary
for i, gal_dict in enumerate(ised_gals): 
    nsa_dict = {} 
    for key in nsa.__dict__.keys(): 
        nsa_dict[key] = getattr(nsa, key)[match_nsa[0][list(match_nsa[1]).index(i)]]
    
    gal_dict['nsa_dict'] = nsa_dict

In [27]:
for i_gal in range(len(ised_gals)): 
    print 'maggies used in iSEDfit: (nMgy)'
    print ised_gals[i_gal]['maggies'][2:]*1e9
    print 'fluxes from SDSS imaging...'
    for key in ised_gals[i_gal]['sdss_dict'].keys(): 
        if key in ['modelflux', 'petroflux', 'fiberflux', 'expflux', 'devflux']: 
            print '      ', key,' ', ised_gals[i_gal]['sdss_dict'][key]
    print 'fluxes from NSA...'
    print '      nmgy ', ised_gals[i_gal]['nsa_dict']['nmgy'][2:]
    for key in ised_gals[i_gal]['nsa_dict'].keys(): 
        if key in ['sersicflux', 'fiberflux', 'petroflux']: 
            print '      ', key,' ', ised_gals[i_gal]['nsa_dict'][key][2:]
    print '================================================='

maggies used in iSEDfit: (nMgy)
[  42.50177002  111.72774506  168.66938782  210.47738647  258.04071045]
fluxes from SDSS imaging...
       modelflux   [  39.40839005  123.764534    186.05612183  235.58732605  283.43804932]
       petroflux   [  34.28339005   92.12915039  140.96487427  178.08909607  204.38491821]
       fiberflux   [  7.87066936  24.68273735  37.02164078  47.52305603  56.63420105]
       expflux   [  26.65555954   83.93719482  125.73600006  156.57980347  185.84898376]
       devflux   [  45.76390839  119.23313904  186.05612183  234.26959229  294.00894165]
fluxes from NSA...
      nmgy  [  42.50177002  111.72774506  168.66938782  210.47738647  258.04071045]
       sersicflux   [  37.88815689  106.37458801  162.9631958   208.94824219  261.51831055]
       fiberflux   [  8.86465931  27.19680977  41.89001465  54.61552429  64.24524689]
       petroflux   [  41.26671219   97.68169403  150.71589661  193.11863708  234.78257751]
maggies used in iSEDfit: (nMgy)
[  183.00810242  1

## maggies used in iSEDfit == NSA_data['nmgy']

In [30]:
for i_gal in range(len(ised_gals)): 
    for key in ised_gals[i_gal]['sdss_dict'].keys(): 
        if 'flux' in key and 'ivar' not in key: 
            ised_gals[i_gal]['sdss_'+key] = ised_gals[i_gal]['sdss_dict'][key]        

In [31]:
# step 1 of pipeline: account for extinction!
# extinction factor
for i_gal in range(len(ised_gals)): 
    ised_gals[i_gal]['f_extinct'] = 10**(0.4 * ised_gals[i_gal]['sdss_dict']['extinction'])
    print ised_gals[i_gal]['f_extinct']

[ 1.23195052  1.16589022  1.11775267  1.08807564  1.06167531]
[ 1.05829394  1.04256964  1.0306977   1.02319181  1.01638842]
[ 1.08509862  1.06193495  1.04454792  1.03360069  1.02370858]


In [32]:
for i_gal in range(len(ised_gals)): 
    for key in ised_gals[i_gal]['sdss_dict'].keys(): 
        if 'flux' in key and 'ivar' not in key: 
            ised_gals[i_gal]['sdss_'+key+'_ext'] = ised_gals[i_gal]['sdss_'+key] * ised_gals[i_gal]['f_extinct']

In [35]:
for i_gal in range(len(ised_gals)): 
    print 'maggies used in iSEDfit (in nMgy): '
    print ised_gals[i_gal]['maggies'] * 1e9
    print 'fluxes from SDSS imaging..., extincted'
    for key in ised_gals[i_gal]['sdss_dict'].keys(): 
        if 'flux' in key and 'ivar' not in key: 
            print ised_gals[i_gal]['sdss_'+key+'_ext']

maggies used in iSEDfit (in nMgy): 
[  11.87881374   15.68270302   42.50177002  111.72774506  168.66938782
  210.47738647  258.04071045]
fluxes from SDSS imaging..., extincted
[  131.45669556   446.36694336   769.90655518  1083.96264648  1399.4420166 ]
[  10.83803368   49.23117065   87.80712891  115.69338989  160.41427612]
[  1.87028551   2.59909701   6.39252234  11.9516964   37.13591003]
[  157.87324524   484.84887695   815.03070068  1137.05444336  1367.6151123 ]
[  1.81147218   2.56632638   6.26586866  11.88701725  36.24541855]
[  20.05649376   66.69441986  125.57663727  185.54875183  246.87739563]
[  134.04081726   467.09365845   769.90637207  1064.18701172  1300.78662109]
[  175.19822693   645.40270996  1037.61865234  1479.36901855  1968.77075195]
maggies used in iSEDfit (in nMgy): 
[  5.70189524e+00   1.45432787e+01   1.83008102e+02   1.10693701e+03
   2.78894702e+03   4.41160742e+03   6.29330713e+03]
fluxes from SDSS imaging..., extincted
[  125.87782288   790.07434082  1898.2564